In [0]:
!wget -c http://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')
!conda install tsnecuda cuda101 -c cannylab -y
!conda install -c plotly plotly=4.5.2 -y
!pip3 install wget instagram-scraper ufal.udpipe

In [0]:
import pandas as pd
import codecs
import json
import os
from tqdm import tqdm

In [66]:
!instagram-scraper sashashpakofficial --comments --maximum 3 --media-types none

Searching sashashpakofficial for posts: 2 media [00:14,  8.53s/ media]Exception ignored in: <bound method tqdm.__del__ of Searching sashashpakofficial for posts: 2 media [00:14,  8.53s/ media]>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 931, in __del__
    self.close()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 1133, in close
    self._decr_instances(self)
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 496, in _decr_instances
    cls.monitor.exit()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py", line 52, in exit
    self.join()
  File "/usr/lib/python3.6/threading.py", line 1053, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread


In [67]:
with codecs.open('./sashashpakofficial/sashashpakofficial.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)
comments = []
for post in raw_data['GraphImages']:
    for comment in post['comments']['data']:
        comments.append(
            {
                'post_id':post['id'],
                'username':comment['owner']['username'],
                'text':comment['text']
            }
        )
data = pd.DataFrame(comments)
print(len(data))
data.head()

1298


,post_id,username,text
0,2259747273458597963,v00000777777,😊😊😊
1,2259747273458597963,nat_viktorov_na,Эт какой же тогда мальчишник был😕
2,2259747273458597963,potehina_polly,@favourite_phoenix Я делаю в иншоте. А так мн...
3,2259747273458597963,a_n_a_s_t_a_s_i_a86,"Тётя и ребенок. Зачем заказывают такие танцы,к..."
4,2259747273458597963,nastya_komentskix,Саша почему не станцевал? У тебя бёдра как у л...


In [0]:
data['text'] = data['text'].str.replace('@\S*','')

In [44]:
!git clone "https://github.com/akutuzov/webvectors.git"
!git clone "https://github.com/ufal/udpipe.git"

fatal: destination path 'webvectors' already exists and is not an empty directory.
fatal: destination path 'udpipe' already exists and is not an empty directory.


In [0]:
from webvectors.preprocessing.rus_preprocessing_udpipe import *

UDPipe model not found. Downloading...

Loading the model...
Processing input...


In [69]:
# URL of the UDPipe model
udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udpipe_filename = udpipe_model_url.split('/')[-1]

if not os.path.isfile(udpipe_filename):
    print('UDPipe model not found. Downloading...', file=sys.stderr)
    wget.download(udpipe_model_url)

print('\nLoading the model...', file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

print('Processing input...', file=sys.stderr)
tokens = []
for line in tqdm(data['text']):
    res = unify_sym(line.strip())
    output = process(process_pipeline, text=res)
    tokens.append(output)


Loading the model...
Processing input...
100%|██████████| 1298/1298 [00:07<00:00, 163.28it/s]


In [0]:
data['tokens'] = tokens

In [0]:
import gensim
import gensim.downloader as api
import numpy as np
import wget

In [71]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/20/184.zip'
#m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning:

This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function



In [0]:
data = data.loc[data['tokens'].apply(lambda row: [x if x in model.vocab else False for x in row] != [False])]

In [0]:
X = []
for row in data['tokens']:
    filt_row = list(filter(lambda x: x in model.vocab, row))
    if filt_row:
        X.append(np.sum([model.get_vector(x) for x in filt_row],axis=0))
    else:
        X.append(np.zeros((300,)))
X = np.array(X)

In [0]:
import plotly.graph_objects as go
from tsnecuda import TSNE

In [77]:
%%time
tsne_representation = TSNE(n_components=2).fit_transform(X)

CPU times: user 521 ms, sys: 271 ms, total: 792 ms
Wall time: 797 ms


In [79]:
fig = go.Figure()
fig.update_layout(width=700,height=700)
fig.add_trace(go.Scatter(x=tsne_representation[:,0], y=tsne_representation[:,1],
                    mode='markers',
                    text=data['text'].str.replace('.', '<br>')
                      ))